### Merging Yellow Green Taxi Trips Data

In [0]:
from pyspark.sql.types import *
from pyspark.sql.functions import *

In [0]:
%run "../include/common_functions"

In [0]:
dbutils.widgets.text("p_file_date","2025-07-01")
v_file_date = dbutils.widgets.get("p_file_date")

In [0]:
v_file_date

In [0]:
year_month = v_file_date[:7]
print(year_month)

#### 1. Loading Taxi Trips Data

In [0]:
silver_folder_path = "abfss://silver@nyctaxitrips2025.dfs.core.windows.net/"

In [0]:
yellow_taxi_df = spark.read.format("delta") \
    .load(f"{silver_folder_path}/yellow_taxi_trips") \
    .filter(col('file_year_month') == f'{year_month}')

In [0]:
green_taxi_df = spark.read.format("delta") \
    .load(f"{silver_folder_path}/green_taxi_trips") \
    .filter(col("file_year_month") == f'{year_month}')

In [0]:
green_taxi_df.show()

In [0]:
all_taxi_trips_df = yellow_taxi_df.union(green_taxi_df)

In [0]:
gold_folder_path = "abfss://gold@nyctaxitrips2025.dfs.core.windows.net/"

In [0]:
all_taxi_trips_df.count()

In [0]:
# all_taxi_trips_df.groupBy("trip_id") \
#     .agg(count("*").alias("count")) \
#     .filter(col("count") > 1) \
#     .orderBy(col("count").desc()) \
#     .show(truncate=False)

In [0]:
all_taxi_trips_df.show()

In [0]:
merge_condition = "tgt.trip_id = src.trip_id AND tgt.file_year_month = src.file_year_month"
merge_delta_data(
    all_taxi_trips_df, 
    'taxi_trips_2025', 
    'gold', 
    'all_taxi_trips', 
    gold_folder_path, 
    merge_condition, 
    "file_year_month")

In [0]:
%sql
select * from taxi_trips_2025.gold.all_taxi_trips
where service_type = "green"
and file_year_month = '2025-08';

In [0]:
dbutils.notebook.exit("Success")